In [ ]:
import pandas as pd
from unidecode import unidecode
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from ast import literal_eval
import time
import re
import fasttext

start_timer = time.time()

class LanguageIdentification:
    def __init__(self):
        pretrained_lang_model = "lang_file.bin"
        self.model = fasttext.load_model(pretrained_lang_model)
    def predict_lang(self, text):
        predictions = self.model.predict(text) 
        return predictions

def to_english(s):
    s = list(s)
    for i in range (len(s)):
        s[i] = unidecode(s[i])
    s1 = ""
    for i in s:
        s1+=i
    return s1
def normalize(s):
    s = s.lower()
    s = " ".join(s.split())
    if s[:6] == "truong":
        s = s[6:]
    s = " ".join(s.split())
    s = s.replace("-"," ")
    s = s.replace("."," ")
    s = s.replace("&"," ")
    s = s.replace("*", " ")
    s = s.replace(","," ")
    s = s.replace("_"," ")
    s = s.replace(" va "," ")
    s = s.replace("cdn", "cao dang nghe")
    s = s.replace("gtvt", "giao thong van tai")
    s = s.replace("thanh pho", " ")
    s = s.replace("bqp", " bo quoc phong ")
    s = s.replace("qg"," quoc gia ")
    s = s.replace("dh", " dai hoc ")
    s = s.replace("bk", " bach khoa ")
    s = s.replace("cd"," cao dang ")
    s = s.replace(" sp "," su pham ")
    s = s.replace(" csnd", " canh sat nhan dan")
    s = s.replace(" cs", " co so ")
    s = s.replace(" cn ", " cong nghe ")
    s = s.replace("khxh", " khoa hoc xa hoi ")
    s = s.replace (" nv ", " nhan van ")
    s = s.replace("tdtt", "the duc the thao")
    s = s.replace("ttgdtx", "trung tam giao duc thuong xuyen")
    s = s.replace("gdtx"," gia duc thuong xuyen ")
    s = s.replace(" hn"," ha noi")
    s = s.replace("ho chi minh","hcm")
    s = s.replace("tp","")
    s = s.replace("ptth", "pho thong trung hoc")
    s = s.replace("hcmc","hcm")
    s = s.replace(" dn"," da nang")
    s = s.replace(" city", " ")
    s = s.replace("city ", " ")
    s = s.replace("cntt"," cong nghe thong tin ")
    s = s.replace("iii", " 3 ")
    s = s.replace("ii"," 2 ")
    s = s.replace("thpt", "trung hoc pho thong")
    s = s.replace("ptth"," pho thong trung hoc")
    s = s.replace("thcs", "trung hoc co so")
    s = re.sub(r'\(.+?\)\s*', '', s)
    s = " ".join(s.split())
    return s
def turn(s):
    return literal_eval(s)
language = LanguageIdentification()
df = pd.read_csv('example.csv')
data = dict()
data["school"] = []
data["school_norm"] = []
data["school_standard"] = []
data["specialization"] = []
data["specialization_norm"] = []
data["specialization_standard"] = []
for ind in df.index:
    data["school"].append(df['school_name'][ind])
    data["school_norm"].append(" ".join(normalize(to_english(str(df['school_name'][ind]))).split()))
    data["specialization"].append(df['specialization'][ind])
    if "trung hoc" in data["school_norm"]:
        data["specialization_norm"].append("trung hoc")
    else:
        data['specialization_norm'].append(" ".join(normalize(to_english(str(df['specialization'][ind]))).split()))
school_vn = pd.read_csv('new 1.txt', sep = "|")
school_en = pd.read_csv('new 1 en.txt', sep = "|")
major_vn = pd.read_csv("major.txt", sep = "|")
major_en = pd.read_csv("major_en.txt", sep = "|")
lst_s = []
lst_m = []
for ind in school_vn.index:
    lst_s.append(" ".join(normalize(to_english(str(school_vn["school"][ind]))).split()))
    school_en["school"][ind] = " ".join(normalize(to_english(str(school_en["school"][ind]))).split())
for ind in major_vn.index:
    lst_m.append(" ".join(normalize(to_english(str(major_vn["specialization"][ind]))).split()))
    major_en["specialization"][ind] = " ".join(normalize(to_english(str(major_en["specialization"][ind]))).split())
school_vn.insert(1,column = "school_vn_norm",value = lst_s)
major_vn.insert(1,column = "major_vn_norm",value = lst_m)
track = 0
track_p = 0
track_r = 0
temp_school = dict()
print("school_begin")
for value in pd.Series(data["school_norm"]).unique():
    temp_school[value] = dict()
    if "trung hoc" in value:
        temp_school[value]["standard"] = "Trung Học"
        temp_school[value]["p_score"] = 100
        temp_school[value]["r_score"] = 100
        continue
    lang = language.predict_lang(str(value))[0][0][9:]
    track = 0
    track_p = 0
    track_r = 0
    start = 0
    end = len(school_vn["school"])
    if lang == "en":
        file = school_en
        norm = "school"
    else:
        file = school_vn
        norm = "school_vn_norm"
    if "dai hoc" in value or "hoc vien" in value or lang == "en":
        end = 297
    if "cao dang" in value:
        start = 297
    for ind_s in range(start,end):
        p = fuzz.token_set_ratio(file[norm][ind_s],value)
        r = fuzz.ratio(file[norm][ind_s],value)
        if p >= track_p and r >= track_r:
            track = ind_s
            track_p = p
            track_r = r
    if track_p >= 90 or track_r >= 95:
        temp_school[value]["standard"] = school_vn["school"][track]
        temp_school[value]["p_score"] = track_p
        temp_school[value]["r_score"] = track_r
    elif "dai hoc" in value or "cao dang" in value or "university" in value:
        temp_school[value]["standard"] = "Not in List"
        temp_school[value]["p_score"] = track_p
        temp_school[value]["r_score"] = track_r

    else:
        temp_school[value]["standard"] = school_vn["school"][track]
        temp_school[value]["p_score"] = track_p
        temp_school[value]["r_score"] = track_r
print(temp_school)
print("major begin")
temp_major = dict()
for value in pd.Series(data["specialization_norm"]).unique():
    temp_major[value] = dict()
    if value == "trung hoc":
        temp_major[value]["standard"] = "Trung Học"
        temp_major[value]["p_score"] = 100
        temp_major[value]["r_score"] = 100
        continue
    if value == "chuyen nganh":
        temp_major[value]["standard"] = "Chuyên Ngành"
        temp_major[value]["p_score"] = 100
        temp_major[value]["r_score"] = 100
        continue
    track = 0
    track_p = 0
    track_r = 0
    lang = language.predict_lang(str(value))[0][0][9:]
    if lang == "en":
        file = major_en
        norm = "specialization"
    else:
        file = major_vn
        norm = "major_vn_norm"
    for ind_s in major_vn.index:
        p = fuzz.token_set_ratio(file[norm][ind_s],value)
        r = fuzz.ratio(file[norm][ind_s],value)
        if p >= track_p and r >= track_r:
            track = ind_s
            track_p = p
            track_r = r
    temp_major[value]["standard"] = major_vn["specialization"][track]
    temp_major[value]["p_score"] = track_p
    temp_major[value]["r_score"] = track_r
print(temp_major)
data["school_p_score"] = []
data["school_r_score"] = []
data["specialization_p_score"] = []
data["specialization_r_score"] = []
for ind_d in range(len(data["school"])):
    if temp_school[data["school_norm"][ind_d]]["standard"] == "Not in List":
        data["school_standard"].append(data["school"][ind_d])
    else:
        data["school_standard"].append(temp_school[data["school_norm"][ind_d]]["standard"])
    data["school_p_score"].append(temp_school[data["school_norm"][ind_d]]["p_score"])
    data["school_r_score"].append(temp_school[data["school_norm"][ind_d]]["r_score"])
    data["specialization_standard"].append(temp_major[data["specialization_norm"][ind_d]]["standard"])
    data["specialization_p_score"].append(temp_major[data["specialization_norm"][ind_d]]["p_score"])
    data["specialization_r_score"].append(temp_major[data["specialization_norm"][ind_d]]["r_score"])
final = pd.DataFrame(data)
final.to_csv("finalized.csv")
print(final)
end_timer = time.time()
print("Execution time :",(end_timer-start_timer), "s")